In [50]:
# IPython magic
%config IPCompleter.greedy=True

# Imports
import pandas as pd
import json
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

# Pandas settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [51]:
dfCols=['map', 
        'CT_score', 
        'T_score',
        'is_overtime',
        'round_status', 
        'round_status_time_left',
        'CT_total_health', 
        'T_total_health',
        'CT_num_alive_players',
        'T_num_alive_players',
        'CT_equip_value',
        'T_equip_value',
        'CT_money',
        'T_money',
        'CT_num_grenades',
        'T_num_grenades',
        'CT_num_he',
        'T_num_he',
        'CT_num_flash',
        'T_num_flash',
        'CT_num_smokes',
        'T_num_smokes',
        'CT_num_molly',
        'T_num_molly',
        'CT_has_Defuser', 
        'bomb_site', 
        'bomb_dropped',
        'num_active_smokes',
        'num_active_molotovs',
        'round_winner']

equipmentValue={
    'Defuser': 400,      # Equipment
    'Kevlar': 650,
    'Helmet': 1000,
    'ZeusX27': 200,
    'P2000': 200,        # Pistols
    'UspS': 200,
    'Glock': 200,
    'P250': 300,
    'FiveSeven': 500,
    'Tec9': 500,
    'Cz75Auto': 500,
    'Elite': 400, #dual berretas
    'Deagle': 700,
    'R8Revolver': 600,
    'Nova': 1050,          # Shotguns
    'Xm1014': 2000,
    'Mag7': 1300,
    'Sawedoff': 1100,
    'Mp9': 1250,           # SMGs
    'Mac10': 1050,
    'Bizon': 1400,
    'Mp7': 1500,
    'Ump45': 1200,
    'P90': 2350,
    'Mp5sd': 1500,
    'Famas': 2050,         # Rifles
    'GalilAr': 1800,
    'M4a4': 3100,
    'M4a1S': 2900,
    'Ak47': 2700,
    'Aug': 3300,
    'Sg553': 3000,
    'Ssg08': 1700,         # Snipers
    'Awp': 4750,
    'Scar20': 5000,
    'G3sg1': 5000,
    'M249': 5200,          # LMGs
    'Negev': 1700,
    'HeGrenade': 300,      # Grenades
    'Flashbang': 200,
    'SmokeGrenade': 300,
    'DecoyGrenade': 50,
    'MolotovGrenade': 400,
    'IncendiaryGrenade': 600,
    'C4': 0,
}

grenadeTypes = {'HeGrenade', 'Flashbang', 'SmokeGrenade', 'DecoyGrenade', 'MolotovGrenade', 'IncendiaryGrenade'}

primaryWeapons = {
    'Nova',          # Shotguns
    'Xm1014',
    'Mag7',
    'Sawedoff',
    'Mp9',           # SMGs
    'Mac10',
    'Bizon',
    'Mp7',
    'Ump45',
    'P90',
    'Mp5sd',
    'Famas',         # Rifles
    'GalilAr',
    'M4a4',
    'M4a1S',
    'Ak47',
    'Aug',
    'Sg553',
    'Ssg08',         # Snipers
    'Awp',
    'Scar20',
    'G3sg1',
    'M249',          # LMGs
    'Negev',
}

secondaryWeapons = {
    'P2000',        # Pistols
    'UspS',
    'Glock',
    'P250',
    'FiveSeven',
    'Tec9',
    'Cz75Auto',
    'Elite',
    'Deagle',
    'R8Revolver',
}

In [52]:
def getEquipmentValue(item):
    itemType = item['item_type']
    if not itemType in equipmentValue:
        print('Warning - item_type not recognised: {}'.format(itemType))
        return None
    if itemType in grenadeTypes:
        return equipmentValue[itemType] * item['clip_ammo']
    else:
        return equipmentValue[itemType]

In [53]:
def getGrenadeNumbers(players, line):
    grenadesCT = grenadesT = heCT = heT = flashCT = flashT = smokeCT = smokeT = mollyCT = mollyT = 0
    
    for player in players:
        if player['team'] == 'CT':
            for item in player['inventory']:
                itemType = item['item_type']
                if not itemType in grenadeTypes:
                    continue
                grenadesCT += item['clip_ammo']
                if (itemType == 'HeGrenade'):
                    heCT += item['clip_ammo']
                if (itemType == 'Flashbang'):
                    flashCT += item['clip_ammo']
                if (itemType == 'SmokeGrenade'):
                    smokeCT += item['clip_ammo']
                if (itemType == 'IncendiaryGrenade'):
                    mollyCT += item['clip_ammo']
        else:
            for item in player['inventory']:
                itemType = item['item_type']
                if not itemType in grenadeTypes:
                    continue
                grenadesT += item['clip_ammo']
                if (itemType == 'HeGrenade'):
                    heT += item['clip_ammo']
                if (itemType == 'Flashbang'):
                    flashT += item['clip_ammo']
                if (itemType == 'SmokeGrenade'):
                    smokeT += item['clip_ammo']
                if (itemType == 'MolotovGrenade'):
                    mollyT += item['clip_ammo']
    
    line['CT_num_grenades'] = grenadesCT
    line['T_num_grenades'] = grenadesT
    line['CT_num_he'] = heCT
    line['T_num_he'] = heT
    line['CT_num_flash'] = flashCT
    line['T_num_flash'] = flashT
    line['CT_num_smokes'] = smokeCT
    line['T_num_smokes'] = smokeT
    line['CT_num_molly'] = mollyCT
    line['T_num_molly'] = mollyT
    
    return line

In [ ]:
def hasPrimaryWeapon(inventory):
    for item in inventory:
        if (item['item_type'] in primaryWeapons)

In [54]:
def processPlayers(screenshot, line):
    totalHealthCT = totalHealthT = numPlayersCT = numPlayersT = equipValueCT = equipValueT = 0
    moneyCT = moneyT = grenadesCT = grenadesT = 0
    line['CT_has_Defuser'] = False
    line['bomb_dropped'] = (line['round_status'] != 'BombPlanted')
    line = getGrenadeNumbers(screenshot['alive_players'], line)
    
    for player in screenshot['alive_players']:
        if player['team'] == 'CT':
            numPlayersCT += 1
            totalHealthCT += player['health']
            moneyCT += player['money']
            if player['has_defuser'] == True:
                line['CT_has_Defuser'] = True
                equipValueCT += equipmentValue['Defuser']
            for item in player['inventory']:
                equipValueCT += getEquipmentValue(item)
            if player['has_helmet']:
                equipValueCT += equipmentValue['Helmet'] * (player['armor'] / 100.0)
            elif player['armor'] > 0:
                equipValueCT += equipmentValue['Kevlar'] * (player['armor'] / 100.0)
        else:
            numPlayersT += 1
            totalHealthT += player['health']
            moneyT += player['money']
            for item in player['inventory']:
                equipValueT += getEquipmentValue(item)
                if item['item_type'] == 'C4' and line['bomb_dropped'] == True:
                    line['bomb_dropped'] = False
            if player['has_helmet']:
                equipValueT += equipmentValue['Helmet']
            elif player['armor'] > 0:
                equipValueT += equipmentValue['Kevlar']
                
    line['CT_total_health'] = totalHealthCT
    line['T_total_health'] = totalHealthT
    line['CT_equip_value'] = equipValueCT
    line['T_equip_value'] = equipValueT
    line['CT_num_alive_players'] = numPlayersCT
    line['T_num_alive_players'] = numPlayersT
    line['CT_money'] = moneyCT
    line['T_money'] = moneyT
    
    return line

In [55]:
# returns the line to add to dataframe after extracting the data
def processScreenshot(screenshot):
    line = {key: None for key in dfCols}
    line['map'] = screenshot['map']
    line['CT_score'] = screenshot['current_score'][0]
    line['T_score'] = screenshot['current_score'][1]
    line['is_overtime'] = (line['CT_score'] >= 15 and line['T_score'] >= 15)
    line['round_status'] = screenshot['round_status']
    line['round_status_time_left'] = screenshot['round_status_time_left']
    line['round_winner'] = screenshot['round_winner']
    if screenshot['planted_bomb'] != None:
        line['bomb_site'] = screenshot['planted_bomb']['site']
    line = processPlayers(screenshot, line)
    
    line['num_active_smokes'] = len(screenshot['active_smokes'])
    line['num_active_molotovs'] = len(screenshot['active_molotovs'])
    
    return line

In [56]:
# Returns False if screenshot is invalid, True otherwise
def validateScreenshot(scrsht): 
    # player counts
    if (scrsht['CT_num_alive_players'] >= 6 or scrsht['T_num_alive_players'] >= 6):
        return False
    if (scrsht['CT_total_health'] > 500 or scrsht['T_total_health'] > 500):
        return False
    
    # normal phase is 115 seconds long, and bomb timer is 40 seconds long
    if (scrsht['round_status_time_left'] > 116): 
        return False
    if ((scrsht['round_status'] == 'BombPlanted') & (scrsht['round_status_time_left'] > 41)):
        return False
    
    
#     if (scrsht['CT_score'] > 16 or scrsht['T_score'] > 16)  #overtime?
#         return False

    # max team money is either 50k (10k per player), or 80k (16k per player), can't know which it is
    if ((scrsht['T_money'] > 80000) | (scrsht['CT_money'] > 80000)):
        return False
    
    # some times in the data players start the game with incredible ammounts of cash, which changes the whole game. 
    if ((scrsht['T_score'] == 0) & (scrsht['CT_score'] == 0) & ((scrsht['T_money'] > 4000) | (scrsht['CT_money'] > 4000))):
        return False

    return True

In [58]:
def loadDataset(datasetFolder, numFiles):
    dataFrame = pd.DataFrame(columns=dfCols)
    rejectedData = []

    fileIndex = 0
    while fileIndex <= numFiles:
        filePath = datasetFolder + "/dataset_" + str(fileIndex).zfill(2) + ".json"
        print(filePath)

        rowsFile = []

        jsonFile = open(filePath)
        data = json.load(jsonFile)
        for index, i in enumerate(data):        
            processedData = processScreenshot(i)
            if not (validateScreenshot(processedData)):
                rejectedData.append(processedData)
                continue

            # DEBUG
            processedData['fileNumber'] = fileIndex
            processedData['indexInFile'] = i['index']
            #processedData['Debug'] = json.dumps(i, separators=(',', ':'))

            rowsFile.append(processedData)

        dfFile = pd.DataFrame(rowsFile)
    #     display(dfFile)
        dataFrame = dataFrame.append(dfFile, ignore_index=True) 

        jsonFile.close()
        fileIndex += 1

    print("Rejected data:")
    dfRejected = pd.DataFrame(rejectedData, columns=dfCols)
    display(dfRejected)
    
    return dataFrame